In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
TRAIN_CSV = "dataset_csv/fer2013_train.csv"
TEST_CSV  = "dataset_csvfer2013_test.csv"
LANDMARK_START_COL = 4

df_train = pd.read_csv(TRAIN_CSV, header=None)
X_train_raw = df_train.iloc[:, LANDMARK_START_COL:].values.astype(np.float32)
y_train = df_train.iloc[:,2].values


In [3]:
# Chuẩn hóa landmarks
def center_scale_landmarks(X):
    X_cs = []
    for s in X:
        xs = s[::2]
        ys = s[1::2]
        cx, cy = xs.mean(), ys.mean()
        xs = xs - cx
        ys = ys - cy
        scale = max(xs.max()-xs.min(), ys.max()-ys.min())
        xs /= scale
        ys /= scale
        X_cs.append(np.column_stack([xs, ys]).flatten())
    return np.array(X_cs)

X_train_cs = center_scale_landmarks(X_train_raw)

# Shape PCA
pca_shape = PCA(n_components=0.95)
X_train_pca = pca_shape.fit_transform(X_train_cs)
scaler = StandardScaler()
X_train_pca = scaler.fit_transform(X_train_pca)

# Train classifier
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train_pca, y_train)

EMOTION_LABELS = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [6]:
def webcam_emotion_demo():
    mp_face = mp.solutions.face_mesh
    cap = cv2.VideoCapture(0)

    with mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face_mesh:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = face_mesh.process(img_rgb)
            
            if result.multi_face_landmarks:
                for face_landmarks in result.multi_face_landmarks:
                    h, w, _ = frame.shape
                    
                    # --- Vẽ landmarks ---
                    for lm in face_landmarks.landmark:
                        x, y = int(lm.x * w), int(lm.y * h)
                        cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    
                    # --- Chuẩn bị landmarks cho classifier ---
                    lms = []
                    for lm in face_landmarks.landmark:
                        lms.append(lm.x)
                        lms.append(lm.y)
                    lms = np.array(lms).reshape(1, -1)
                    lms_cs = center_scale_landmarks(lms)
                    lms_pca = pca_shape.transform(lms_cs)
                    lms_pca = scaler.transform(lms_pca)
                    
                    # --- Dự đoán emotion ---
                    proba = clf.predict_proba(lms_pca)[0]  # xác suất từng lớp
                    pred_id = np.argmax(proba)
                    pred_emotion = EMOTION_LABELS[pred_id]
                    pred_conf = proba[pred_id]
                    
                    # --- Hiển thị ---
                    text = f"{pred_emotion} ({pred_conf*100:.1f}%)"
                    cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)
            
            cv2.imshow("Emotion + Landmarks", frame)
            if cv2.waitKey(1) & 0xFF == 27:  # Nhấn ESC để thoát
                break

    cap.release()
    cv2.destroyAllWindows()

In [7]:
if __name__ == "__main__":
    webcam_emotion_demo()

In [ ]:
# mp_face = mp.solutions.face_mesh
# cap = cv2.VideoCapture(0)

# with mp_face.FaceMesh(static_image_mode=False, max_num_faces=1) as face_mesh:
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         result = face_mesh.process(img_rgb)
        
#         if result.multi_face_landmarks:
#             for face_landmarks in result.multi_face_landmarks:
#                 # Draw landmarks
#                 h, w, _ = frame.shape
#                 for lm in face_landmarks.landmark:
#                     x, y = int(lm.x * w), int(lm.y * h)
#                     cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                
#                 # Prepare landmarks for classifier
#                 lms = []
#                 for lm in face_landmarks.landmark:
#                     lms.append(lm.x)
#                     lms.append(lm.y)
#                 lms = np.array(lms).reshape(1, -1)
#                 lms_cs = center_scale_landmarks(lms)
#                 lms_pca = pca_shape.transform(lms_cs)
#                 lms_pca = scaler.transform(lms_pca)
                
#                 pred_id = clf.predict(lms_pca)[0]
#                 pred_emotion = EMOTION_LABELS[pred_id]
#                 proba = clf.predict(lms_pca)[0] 
#                 pred_conf = proba[pred_id]  # xác suất dự đoán
#                 # Vẽ text
#                 cv2.putText(frame, pred_emotion, (50,50), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)
        
#         cv2.imshow("Emotion + Landmarks", frame)
#         if cv2.waitKey(1) & 0xFF == 27:  # ESC để thoát
#             break

# cap.release()
# cv2.destroyAllWindows()


IndexError: invalid index to scalar variable.